In [1]:
import os
import datetime
import pprint
import xlrd
import dataset

In [2]:
def dated_file_name(extension, prefix):
    filename = prefix + "_" + datetime.datetime.now().isoformat() + "." + extension
    return filename

In [3]:
data_dir = "/Users/jjpr/dev/scratch/mkgu_scratch/data"

In [4]:
class MappingFile(object):
    def __repr__(self):
        return str((self.__class__.__name__, vars(self)))

In [5]:
mf_paths = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if x.endswith(".xls")]
mf_paths

['/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-4F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5H.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-8E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-5-SN0656.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-8-SN0657.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2466-1-SN0658.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2467-6_SN0649.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/5602-2 SN1025-000936.xls']

In [6]:
def is_comment_row(row):
    return str(row[0].value).lstrip().startswith("//")

In [7]:
def row_col_valid(cell):
    if cell.ctype != 2: return False
    if type(cell.value) != float: return False
    if cell.value % 1 != 0: return False
    if cell.value < 0: return False
    if cell.value > 9: return False
    return True

In [8]:
def get_mapping(i, row):
    if not (row_col_valid(row[0]) and row_col_valid(row[1])):
        return None
    result = {
        "electrode_column": int(row[0].value), 
        "electrode_row": int(row[1].value), 
        "bank_name": row[2].value,
        "electrode_number_in_bank": row[3].value,
        "electrode_label": row[4].value,
        "index_in_mapping_sheet": i
    }
    return result

In [9]:
def get_array_id(row):
    stripped = row[0].value.strip()
    chopped = stripped[len("Cerebus mapping for array "):]
    squoze = ''.join(chopped.split())
    return squoze

In [10]:
def get_mapping_file(path):
    mf = MappingFile()
    mf.xls_path = path
    mf.workbook = xlrd.open_workbook(path)
    mf.mapping_worksheet = mf.workbook.sheet_by_name("Cerebus mapping")
    mf.comment_lines = set()
    mf.description_row = None
    mf.mappings = []
    for i, row in enumerate(mf.mapping_worksheet.get_rows()):
        if is_comment_row(row):
            mf.comment_lines.add(i)
        elif mf.description_row is None:
            mf.description_row = row
        else:
            mapping = get_mapping(i, row)
            if mapping is not None:
                mf.mappings.append(mapping)
    mf.array_id = get_array_id(mf.description_row)
    return mf

In [11]:
mfs = []

for path in mf_paths:
    mf = get_mapping_file(path)
    mfs.append(mf)

pprint.pprint([vars(mf) for mf in mfs])
# mfs

[{'array_id': '090609B-4F/SN0559',
  'comment_lines': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
  'description_row': [text:'Cerebus mapping for array 090609B-4F/SN0559',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      text:'elec',
                      empty:''],
  'mapping_worksheet': <xlrd.sheet.Sheet object at 0x110d2db38>,
  'mappings': [{'bank_name': 'A',
                'electrode_column': 8,
                'electrode_label': 'elec99',
                'electrode_number_in_bank': '14',
                'electrode_row': 9,
                'index_in_mapping_sheet': 13},
               {'bank_name': 'A',
                'electrode_column': 7,
                'electrode_label': 'elec98',
                'electrode_number_in_bank': '10',
                'electrode_row': 9,
                'index_in_mapping_sheet': 14},
               {'bank_name': 'A',
   

In [12]:
[x.array_id for x in mfs]

['090609B-4F/SN0559',
 '090609B-5E/SN00547',
 '090609B-5F/SN0548',
 '090609B-5H/SN0549',
 '090609B-8E/SN0552',
 '2132-5/SN0656',
 '2132-8/SN0657',
 '2466-1/SN0658',
 '2467-6/SN0649',
 '5602-2/SN1025-000936']

In [13]:
try: 
    db
except NameError:
    db_file = os.path.join(data_dir, dated_file_name("db", "array_meta"))
    db_url = "sqlite:///" + db_file
    db = dataset.connect(db_url)

In [14]:
db

<Database(sqlite:////Users/jjpr/dev/scratch/mkgu_scratch/data/array_meta_2018-02-05T14:53:40.499753.db)>

In [15]:
tbl_array = db["array"]
tbl_mapping = db["mapping"]
for mf in mfs:
    array_dict = {"blackrock_id_from_xls": mf.array_id, "mapping_xls":mf.xls_path}
    array_id = tbl_array.upsert(array_dict, keys=["blackrock_id_from_xls"])
    if type(array_id) == bool:
        array_id = tbl_array.find_one(blackrock_id_from_xls=mf.array_id)["id"]
    print(array_id)
    for mapping in mf.mappings:
        insert = dict(mapping)
        insert["array_id"] = array_id
        tbl_mapping.upsert(insert, keys=["array_id", "electrode_column", "electrode_row"])

1
2
3
4
5
6
7
8
9
10


In [161]:
# tbl_mapping.drop()

In [16]:
animals = [
    {
        "name": "Tito",
        "implants": [
            {"hemisphere": "L", "position": "A", "blackrock_id_from_surgery_note": "2132-8/SN0657",
            "info_source": "/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "M", "blackrock_id_from_surgery_note": "2467-6/SN0649",
            "info_source": "/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "P", "blackrock_id_from_surgery_note": "2132-5/SN0656",
            "info_source": "/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages"},
        ]
    },
    {
        "name": "Chabo",
        "implants": [
            {"hemisphere": "L", "position": "A", "blackrock_id_from_surgery_note": "SN1024-0547",
            "info_source": "/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "M", "blackrock_id_from_surgery_note": "SN1025-0549",
            "info_source": "/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "P", "blackrock_id_from_surgery_note": "SN1025-0552",
            "info_source": "/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery_note.pages"},
        ]
    }
]

In [19]:
# I am just guessing.
blackrock_id_map = [
    {"blackrock_id_from_surgery_note": "2132-8/SN0657", "blackrock_id_from_xls": "2132-8/SN0657"},
    {"blackrock_id_from_surgery_note": "2467-6/SN0649", "blackrock_id_from_xls": "2467-6/SN0649"},
    {"blackrock_id_from_surgery_note": "2132-5/SN0656", "blackrock_id_from_xls": "2132-5/SN0656"},
    {"blackrock_id_from_surgery_note": "SN1024-0547", "blackrock_id_from_xls": "090609B-5E/SN00547"},
    {"blackrock_id_from_surgery_note": "SN1025-0549", "blackrock_id_from_xls": "090609B-5H/SN0549"},
    {"blackrock_id_from_surgery_note": "SN1025-0552", "blackrock_id_from_xls": "090609B-8E/SN0552"}
]

In [17]:
tbl_animal = db["animal"]
tbl_implant = db["implant"]
for animal in animals:
    animal_id = tbl_animal.upsert({"name": animal["name"]}, keys=["name"])
    if type(animal_id) == bool:
        animal_id = tbl_animal.find_one(name=animal["name"])["id"]
    print(animal_id)
    for implant in animal["implants"]:
        id_map = next(x for x in blackrock_id_map if x["blackrock_id_from_surgery_note"]==implant["blackrock_id_from_surgery_note"])
        array_id = tbl_array.find_one(blackrock_id_from_xls=id_map["blackrock_id_from_xls"])["id"]
        insert = {
            "animal_id": animal_id, 
            "array_id": array_id, 
            "hemisphere": implant["hemisphere"], 
            "position": implant["position"],
            "info_source": implant["info_source"],
            "blackrock_id_from_surgery_note": implant["blackrock_id_from_surgery_note"]
        }
        tbl_implant.upsert(insert, keys=["animal_id", "array_id"])

1


KeyError: 'blackrock_id'

In [184]:
([x.array_id for x in mfs], [], [((animal["name"], implant["position"], implant["hemisphere"], implant["blackrock_id"]), implant["info_source"], []) for animal in animals for implant in animal["implants"]])

(['090609B-4F/SN0559',
  '090609B-5E/SN00547',
  '090609B-5F/SN0548',
  '090609B-5H/SN0549',
  '090609B-8E/SN0552',
  '2132-5/SN0656',
  '2132-8/SN0657',
  '2466-1/SN0658',
  '2467-6/SN0649',
  '5602-2/SN1025-000936'],
 [],
 [(('Tito', 'A', 'L', '2132-8/SN0657'),
   '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages',
   []),
  (('Tito', 'M', 'L', '2467-6/SN0649'),
   '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages',
   []),
  (('Tito', 'P', 'L', '2132-5/SN0656'),
   '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages',
   []),
  (('Chabo', 'A', 'L', 'SN1024-0547'),
   '/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery

In [173]:
animals

[{'implants': [{'blackrock_id': '2132-8/SN0657',
    'hemisphere': 'L',
    'info_source': '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages',
    'position': 'A'},
   {'blackrock_id': '2467-6/SN0649',
    'hemisphere': 'L',
    'info_source': '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages',
    'position': 'M'},
   {'blackrock_id': '2132-5/SN0656',
    'hemisphere': 'L',
    'info_source': '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages',
    'position': 'P'}],
  'name': 'Tito'},
 {'implants': [{'blackrock_id': 'SN1024-0547',
    'hemisphere': 'L',
    'info_source': '/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/su